In [ ]:
import tensorflow as tf

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
# Training image preprocessing
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

In [ ]:
# validation image preprocessing
validation_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

In [ ]:
for x,y in training_set:
    print(x,x.shape)
    print(y,y.shape)
    break


In [ ]:
## To avoid overshooting
#1. chooe small learning rate default 0.001 we are taking 0.0001
#2. there may be chance of underfitting , so increase number of neuron
# we have used dropout to avoid overfitting
#3. Add more convolution layer to extract more feature from images there may be possibility that model unable to capture relevant feature or model is confusing due to lack of feature so feed with more feature

In [ ]:
# Building mode
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()

In [ ]:
# building convolution layer

model.add(Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=(128,128,3)))
model.add(Conv2D(filters=32,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.summary()

In [ ]:

model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=64,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.summary()

In [ ]:

model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=128,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.summary()

In [ ]:

model.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=256,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.summary()

In [ ]:

model.add(Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=512,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Dropout(0.25)) # to avoid overfitting

In [ ]:
model.add(Flatten())

In [ ]:
model.summary()

In [ ]:
model.add(Dropout(0.4))

In [ ]:
model.add(Dense(units=1500,activation='relu'))

In [ ]:
# output layer
model.add(Dense(units=38,activation='softmax'))

In [ ]:
# Compiling model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Training model
training_history = model.fit(x=training_set,validation_data=validation_set,epochs=10)


In [ ]:
# model evaluation on Training set
train_loss, train_acc = model.evaluate(training_set)

In [ ]:
print(train_loss, train_acc)

In [ ]:
# Model on validation set
val_loss, val_acc = model.evaluate(validation_set)

In [ ]:
print(val_loss, val_acc)

In [ ]:
# Saving Model
model.save("traine_model.keras")

In [ ]:
training_history.history

In [ ]:
# recording history in json
import json
with open("training_hist.json","w") as f:
    json.dump(training_history.history,f)

In [ ]:
# Accuracy Visualization
epochs = [i for i in range(1,11)]
plt.plot(epochs,training_history.history['accuracy'],color='red',label='Training Accuracy')
plt.plot(epochs,training_history.history['val_accuracy'],color='blue',label='Validation Accuracy')
plt.xlabel('No. of epochs')
plt.ylabel('Accuracy Result')
plt.title('visualization')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs,training_history['accuracy'])

In [ ]:
# some other metrics for model evaluation 

In [ ]:
class_name = validation_set.class_names
class_name

In [ ]:
test_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

In [ ]:
y_pred = model.predict(test_set)
y_pred,y_pred.shape

In [ ]:
# it will go inside this y_pred and it will return the index of max value from it and axis =1 means return the index in columnwise
predicted_categories = tf.argmax(y_pred,axis=1)

In [ ]:
predicted_categories

In [ ]:
true_category = tf.concat([y for x,y in test_set],axis=0)
true_categories

In [ ]:
Y_true = tf.argmax(true_categories,axis=1)
Y_true

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix


In [ ]:
print(classification_report(Y_true,predicted_categories,target_names=class_name))

In [ ]:
cm = confusion_matrix(Y_true,predicted_categories)
cm.shape

In [ ]:
# confusion matrix visualization
plt.figure(figsize=(40,40))
sns.heatmap(cm,annot=True,annot_kws={'size':10})
plt.xlabel('predicted class',fontsize=20)
plt.ylabel('Actual class',fontsize=20)
plt.title('plant disease prediction confusion matrix')
plt.show()